In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

In [ ]:
## Some library
from address_convertor import pubkey_to_address, get_pubkey
from time_manager import get_time

In [ ]:
# preparation database
import sqlite3

path = 'index.db'
conn = sqlite3.connect(path)
cur = conn.cursor()

def create_meta_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Meta (
                     key TEXT PRIMARY KEY,
                     value TEXT);
                ''')

def create_txin_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxIn (
                     tx INTEGER,
                     n INTEGER,
                     addr INTEGER,
                     UNIQUE (tx, n, addr));
                ''')

def create_txout_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxOut (
                     tx INTEGER,
                     n INTEGER,
                     addr INTEGER,
                     UNIQUE (tx, n, addr));
                ''')

def insert_txins(iaddr):
    cur.executemany('''INSERT OR IGNORE INTO TxIn (
                       tx, n, addr) VALUES (
                       ?, ?, ?);
                    ''', iaddr)

def insert_txouts(oaddr):
    cur.executemany('''INSERT OR IGNORE INTO TxOut (
                       tx, n, addr) VALUES (
                       ?, ?, ?);
                    ''', oaddr)
    
def txn2addrid(tx, n):
    cur.execute('''SELECT addr FROM TxOut WHERE
                       tx = ? AND n = ?
                ''', (tx, n))
    return cur.fetchone()[0]

def update_meta(key, value):
    cur.execute('''INSERT OR IGNORE INTO Meta (
                        key, value) VALUES (
                        ?, ?);
                ''', (key, value))
    cur.execute('''UPDATE Meta SET value = ? WHERE key = ?;
                ''', (value, key))

def get_blkid(blkhash):
    cur.execute('''SELECT id FROM BlkID WHERE blkhash = ?''', (blkhash,))
    return cur.fetchone()[0]
    
def get_txid(txhash):
    cur.execute('''SELECT id FROM TxID WHERE txhash = ?''', (txhash,))
    return cur.fetchone()[0]
    
def get_addrid(addr):
    cur.execute('''SELECT id FROM AddrID WHERE addr = ?''', (addr,))
    return cur.fetchone()[0]
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')
    
def journal_mode(mode):
    sql = f'PRAGMA journal_mode = {mode}'
    cur.execute(sql)
    conn.commit()

def synchronous(mode):
    sql = f'PRAGMA synchronous = {mode}'
    cur.execute(sql)
    conn.commit()
    
def get_max_height():
    cur.execute('''SELECT MAX(id) FROM BlkID;''')
    return cur.fetchone()[0] 
    
def get_meta(key):
    cur.execute('''SELECT value FROM Meta WHERE key = ?''', (key,))
    result = cur.fetchone()
    if result is not None:
        result = result[0]
    return result

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 60*5

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

In [ ]:
create_meta_table()
create_txin_table()
create_txout_table()

In [ ]:
def get_addresses_in(theight):
    iaddresses = list()
    block_hash = rpc_connection.getblockhash(theight)
    block = rpc_connection.getblock(block_hash, 2)
    for tx in block['tx']:
        txid = get_txid(tx['txid']) # IMPORTRANT! We should to use txid not hash
        n = -1
        for vin in tx['vin']:
            n = n + 1
            if 'coinbase' in vin.keys():
                iaddresses.append((txid, n, 0))
            else:
                ptxid = get_txid(vin['txid'])
                addrid = txn2addrid(ptxid, vin['vout'])
                iaddresses.append((txid, n, addrid))
    return iaddresses

def get_addresses_out(theight):
    addrs = list()
    block_hash = rpc_connection.getblockhash(theight)
    block = rpc_connection.getblock(block_hash, 2)
    for tx in block['tx']:
        txid = get_txid(tx['txid']) # IMPORTRANT! We should to use txid not hash
        n = -1
        for vout in tx['vout']:
            n = n + 1
            if vout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                'witness_v0_keyhash', 'witness_v0_scripthash',
                                                'witness_unknown', 'multisig'):
                for addr in vout['scriptPubKey']['addresses']:
                    addrid = get_addrid(addr)
                    addrs.append((txid, n, addrid))
            elif vout['scriptPubKey']['type'] in ('pubkey', 'nonstandard', 'nulldata'):
                try:
                    for addr in [pubkey_to_address(get_pubkey(vout['scriptPubKey']['hex']))]:
                        addrid = get_addrid(addr)
                        addrs.append((txid, n, addrid))
                except UnboundLocalError:
                    for addr in [f'{tx["txid"]}{n}']:
                        addrid = get_addrid(addr)
                        addrs.append((txid, n, addrid))
            else:
                raise Exception(f'BUG!! {theight}\n{tx}\n{vout}')
    return addrs

In [ ]:
import time
import pickle
import itertools
import multiprocessing

if os.path.exists('taking_txouthash.pickle'):
    with open('taking_txouthash.pickle', 'rb') as f:
        taking = pickle.load(f)
else:
    taking = dict()

term = 10000
start_height = get_meta('txout_height')
if start_height is not None:
    start_height = int(start_height)
if start_height is None or start_height < term:
    start_height = 0
else:
    start_height = (start_height//term-1)*term
end_height = get_max_height()-1 # default
print(f'Start from {start_height} to {end_height}')

pool_num = multiprocessing.cpu_count()
rpc_connection = get_rpc()
try:
    synchronous('NORMAL')
    journal_mode('WAL')
    for sheight, eheight in zip(range(start_height, end_height, term), 
                                range(start_height+term, end_height+term, term)):
        stime = time.time()
        begin_transactions()
        if eheight >= end_height:
            eheight = end_height+1
        with multiprocessing.Pool(pool_num) as p:
            try:
                results = p.imap(get_addresses_out, range(sheight, eheight))
                for addrs in results:
                    insert_txouts(addrs)
            except KeyboardInterrupt:
                print(f'KeyboardInterrupt occured. Send terminate to processes')
                p.terminate()
                p.join()
                raise KeyboardInterrupt
        update_meta('txout_height', eheight-1)
        commit_transactions()
        etime = time.time()
        if sheight not in taking.keys():
            taking[sheight] = etime-stime
        print(f'TxOut Job done from {sheight} to {eheight-1} during {etime-stime}')
except KeyboardInterrupt:
    print(f'KeyboardInterrupt detected. Commit transactions.')
    try:
        commit_transactions()
    except sqlite3.OperationalError:
        pass
finally:
    try:
        commit_transactions()
    except sqlite3.OperationalError:
        pass
    with open('taking_txouthash.pickle', 'wb') as f:
        pickle.dump(taking, f)
    conn.close()